In [2]:
from sklearn.datasets import fetch_openml   # Fetch dataset from openml

In [3]:
mnist = fetch_openml("mnist_784", version=1, cache=True, as_frame=False)# mnist 데이터 가져오기. 28 * 28 = 784개의 속성 

In [13]:
# 학습데이터, 타겟 데이터 
X = mnist.data
y = mnist.target

In [14]:
import torch   # pytorch 
from torch.utils.data import TensorDataset, DataLoader  # data loading utility
from sklearn.model_selection import train_test_split  # Split arrays or matrices into random train and test subsets

In [15]:
# proportion of the dataset to include in the train split : 1/7 , random_state: None 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=1/7, random_state=0)
# multi-dimensional matrix containing elements of a single data type
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
# 64-bit integer (signed)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [16]:
import torch.nn as nn  # neural network 
import torch.nn.functional as F # dropout, relu 
from torch import optim # optimization algorithms
from torch.autograd import Variable # automatic differentiation

In [17]:
X_train = X_train.view(-1, 1, 28,28).float() # reshape
X_test = X_test.view(-1, 1, 28,28).float()  # reshape 
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [18]:
train = TensorDataset(X_train, y_train)  # training data
test = TensorDataset(X_test, y_test)  # test data 
BATCH_SIZE = 16
loader_train = DataLoader(train, batch_size= BATCH_SIZE, shuffle=False) # batch size = 16, no shuffle 
loader_test = DataLoader(test, batch_size= BATCH_SIZE, shuffle=False)  # batch size = 16, no shuffle 


In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( 
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32), 
            nn.Conv2d(32, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(128, 10)
        )

        self.loss_fn = nn.CrossEntropyLoss() # loss function. CrossEntropy
        self.optimizer = optim.Adam(self.parameters(), lr=0.001) # adam optimizer. learning rate : 0.001

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x
    

In [20]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters()) # use Adam optimizer 
    error = nn.CrossEntropyLoss() # use Cross entropy 
    EPOCHS = 100 # 에포크 
    model.train() # 모델 학습 
    for epoch in range(EPOCHS): # EPOCHS 만큼 반복 
        correct = 0 # 초기화 
        for batch_idx, (X_batch, y_batch) in enumerate(loader_train): # loader_train 순회 
            var_X_batch = Variable(X_batch).float() # PyTorch Tensor의 Wrapper
            var_y_batch = Variable(y_batch)  # PyTorch Tensor의 Wrapper
            optimizer.zero_grad() # 그레디언트 초기화 
            output = model(var_X_batch)  # 모델 적용 
            loss = error(output, var_y_batch) # 손실
            loss.backward() # 손실 역전파 
            optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수를 조정
            predicted = torch.max(output.data, 1)[1]   #  return max_indices
            correct += (predicted == var_y_batch).sum() # correct count 
            if batch_idx % 200 == 0:
                print('에포크 : {} [{}/{} ({:.0f}%)]\t 손실함수 : {:.6f}\t Accuracy:{:.3f}%'\
                      .format(epoch, batch_idx*len(X_batch),len(loader_train),\
                        100.*batch_idx / len(loader_train),loss.data,correct*100./ (BATCH_SIZE*(batch_idx + 1))))


In [ ]:
def evaluate(model): # 모델 평가 
    correct = 0
    for test_imgs, test_labels in loader_test:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        # 가장 높은 값을 가진 인덱스:  예측값
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) / (len(loader_test)*BATCH_SIZE)))

cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)
index = 10
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{} 번째 학습데이터의 테스트 결과 : {}'.format(index,output))
_, predicted = torch.max(output, 1)
print('{} 번째 데이터의 예측측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))